# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [1]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'panel'

In [ ]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [ ]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("./Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")
#sfo_data.head()

file_path = Path("./Data/neighborhoods_coordinates.csv")
neighborhood_locations_df = pd.read_csv(file_path)
#neighborhood_locations_df.head()

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [ ]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""

    sfo_data_mean_of_all_columns = sfo_data.groupby(["year"]).mean()
    sfo_data_housingunits = sfo_data_mean_of_all_columns.drop(['sale_price_sqr_foot', 'gross_rent'],axis=1)
    sfo_data_housingunits.rename(columns={"housing_units": "Mean of housing units"}, inplace=True)
    max_units = sfo_data_housingunits["Mean of housing units"].max()
    min_units = sfo_data_housingunits["Mean of housing units"].min()
    std_dev_units = sfo_data_housingunits["Mean of housing units"].std()
    # Plot the data
    fig_housing_units = plt.figure()
    plot_housing_units = sfo_data_housingunits.plot.bar()
    plot_housing_units.set_xlabel("Year", fontsize=12)
    plot_housing_units.set_ylabel("Housing Units", fontsize=12)
    plot_housing_units.set_title( "Housing Units in San Francisco from 2010 to 2016", fontsize=14, fontweight="bold")
    plot_housing_units.set_ylim(min_units - std_dev_units,  max_units + std_dev_units 
    plt.close(fig_housing_units)

    return pn.pane.Matplotlib(fig_housing_units, tight=True)


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    average_sp_gr_peryear_df = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby('year').mean()
    fig_avg_gross_rent = plt.figure()
    plot_avg_gross_rent = average_sp_gr_peryear_df["gross_rent"].plot()
    plot_avg_gross_rent.set_xlabel("Year", fontsize=12)
    plot_avg_gross_rent.set_ylabel("Gross Rent", fontsize=12)
    plot_avg_gross_rent.set_title(
    "Average Gross Rent in San Francisco", fontsize=14, fontweight="bold"
    )
    plt.close(fig_avg_gross_rent)
    
    return pn.pane.Matplotlib(fig_avg_gross_rent, tight=True)


def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    average_sp_gr_peryear_df = sfo_data[['sale_price_sqr_foot','gross_rent']].groupby('year').mean()
    fig_avg_sale_price = plt.figure()
    plot_avg_sale_price = average_sp_gr_peryear_df["sale_price_sqr_foot"].plot()
    plot_avg_sale_price.set_xlabel("Year", fontsize=12)
    plot_avg_sale_price.set_ylabel("Sale Price per Sft", fontsize=12)
    plot_avg_sale_price.set_title(
    "Average Sale Price per Sft in San Francisco", fontsize=14, fontweight="bold"
    )
    plt.close(fig_avg_sale_price)
    
    return pn.pane.Matplotlib(fig_avg_sale_price, tight=True)


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    average_price_by_nhood_df = sfo_data.groupby(["year", "neighborhood"]).mean()
    average_price_by_nhood_df.reset_index(inplace=True)
    average_price_by_nhood_df.rename(columns={"level_0": "year"}, inplace=True)
    
    fig_average_price_by_neighborhood = plt.figure()
    avg_price_by_neighborhood_plot = average_price_by_nhood_df.hvplot.line(
        "year",
        "sale_price_sqr_foot",
        xlabel= "Year",
        ylabel= "Avg. Sale Price per Square Foot",
        groupby= "neighborhood")
    plt.close(fig_average_price_by_neighborhood)
    return pn.pane.Matplotlib(fig_average_price_by_neighborhood, tight=True)

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    # YOUR CODE HERE!
    neighborhood_df = sfo_data.groupby(['neighborhood']).mean()
    expensive_neighborhood = neighborhood_df.sort_values("sale_price_sqr_foot", ascending = False).head(10)
    expensive_neighborhood.reset_index(drop=False)

    expensive_neighborhood_hvplot =expensive_neighborhood.hvplot.bar(
        "neighborhood",
        "sale_price_sqr_foot",
        title="Top 10 Expensive Neighborhoods in SFO",
        xlabel="Neighborhood",
        ylabel="Avg. Sale Price per Square Foot",
        height=400,
        rot=90
    )
    return expensive_neighborhood_hvplot

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Prices of Most Expensive Neighborhoods."""   
    
    # YOUR CODE HERE!
    neighborhood_df = sfo_data.groupby(['neighborhood']).mean()
    expensive_neighborhood = neighborhood_df.sort_values("sale_price_sqr_foot", ascending = False).head(10)
    average_price_by_nhood_df = sfo_data.groupby(["year", "neighborhood"]).mean()
    average_price_by_nhood_df.reset_index(inplace=True)
    result = pd.merge(expensive_neighborhood, average_price_by_nhood_df,how='inner', on=['neighborhood'])

    result_hvplot = result.hvplot.bar(
        x="neighborhood",
        y=['sale_price_sqr_foot_y','gross_rent_y'],
        title="Top 10 Expensive Neighborhoods in SFO",
        xlabel="Neighborhood",
        ylabel="Housing Units",
        height=400,
        rot=90,
        groupby='neighborhood'
    )
    return result_hvplot


def neighborhood_map():
    """Neighborhood Map."""

    # YOUR CODE HERE!
    sfo_data_mean_of_all_columns = sfo_data.groupby(["year"]).mean()
    neighbourhood_meanvalues_df = sfo_data.groupby(["neighborhood"]).mean()
    neighbourhood_meanvalues_df.reset_index(inplace=True)
    neighbourhood_meanvalues_df.rename(columns={"neighborhood": "Neighborhood"}, inplace=True)
    #neighbourhood_meanvalues_df.head()
    merged_neighbourhood_df = pd.merge(
        neighborhood_locations_df, neighbourhood_meanvalues_df, on="Neighborhood"
        )
    map = px.scatter_mapbox(
        merged_neighbourhood_df,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=15,
        zoom=11,
        hover_name="Neighborhood",
        title="Average values per neighbourhood Scatter Plot Visualisation",
    )
    plotly_panel = pn.pane.Plotly(map)
    plotly_panel._updates = True
    return plotly_panel


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [ ]:
# Create a Title for the Dashboard
# YOUR CODE HERE!

title = pn.pane.Markdown(
    """
# Rental Analysis of San Francisco from 2010 to 2016
""",
    width=800,
)
welcome = pn.pane.Markdown(
    """
This dashboard presents a visualisation of rental analysis of historical prices of house units, sale price per square foot and gross rent in San Francisco from 2010 to 2016. You can navigate through the tabs  to explore details
"""
)

# Create a tab layout for the dashboard
tabs = pn.Tabs(
    ("Welcome", pn.Column(welcome, neighborhood_map())),
    ("Yearly Market Analysis", pn.Column(housing_units_per_year(), average_gross_rent(), average_sales_price())),
    ("Neighborhood Analysis", pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods())),
)


## Serve the Panel Dashboard

In [ ]:
# Serve the# dashboard
# YOUR CODE HERE!
tabs.servable()

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [ ]:
# housing_units_per_year()

In [ ]:
# average_gross_rent()

In [ ]:
# average_sales_price()

In [ ]:
# average_price_by_neighborhood()

In [ ]:
# top_most_expensive_neighborhoods()

In [ ]:
# most_expensive_neighborhoods_rent_sales()

In [ ]:
# neighborhood_map().show()

In [ ]:
# parallel_categories()

In [ ]:
# parallel_coordinates()

In [ ]:
# sunburst()